# LAB7 Assignment
> The document description are designed by JIa Yanhong in 2022. Oct. 20th
------

## LAB Assignment
### Exercise 1 logistic regression (20 points )
This exercise uses dataset digit01.csv , which has 13 columns, and the last column is the dependent variable. 

This part requires you to implement a `logistic regression` using the pytorch framework (defining a logistic regression class that inherits `nn.module`). To test your model, we provide a dataset `digit01.csv` which is in the **datasets folder**. This dataset requires you to divide the training set and the test set by yourself, and it is recommended that 80% of the training set and 20% of the test set be used.

In [32]:
import torch

device = torch.device('mps')  # apple silicon gpu
# device = torch.device('cpu')

+ load datasets

In [33]:
import pandas as pd

digit01 = pd.read_csv('datasets/digit01.csv', header=None)
digit01.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1,1,1,1,0,1,1,0,1,1,1,1,0
1,0,1,1,1,0,1,1,0,1,1,1,1,0
2,1,1,0,1,0,1,1,0,1,1,1,1,0
3,1,1,1,1,0,1,1,0,1,1,1,0,0
4,1,1,1,1,0,1,1,0,1,0,1,1,0


+ Splitting dataset into 80% Training and 20% Testing Data:

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(digit01.iloc[:, :-1], digit01.iloc[:, -1], test_size=0.2, random_state=42)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((51, 12), (13, 12), (51,), (13,))

In [35]:
from torch import from_numpy

X_train = from_numpy(X_train.values).float().to(device)
y_train = from_numpy(y_train.values).long().to(device)
X_test = from_numpy(X_test.values).float().to(device)
y_test = from_numpy(y_test.values).long().to(device)

In [36]:
X_train

tensor([[1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 0.],
        [1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 1.],
        [0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1.],
        [0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0.],
        [0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1.],
        [1., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1.],
        [1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 1., 1.],
        [0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.],
        [0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 1., 0.],
        [1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.],
        [1., 1., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1.],
        [1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1.],
        [0., 0

+ Define a LogisticRegression subclass of nn. Module

In [37]:
from torch import nn

class LogisticRegression(nn.Module):
    def __init__(self, i_dim, o_dim):
        super(LogisticRegression, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(i_dim, o_dim),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.model.forward(x)

+ Create the model

In [38]:
logistic_reg = LogisticRegression(X_train.shape[1], 2).to(device)
logistic_reg.requires_grad_()

LogisticRegression(
  (model): Sequential(
    (0): Linear(in_features=12, out_features=2, bias=True)
    (1): Sigmoid()
  )
)

 + Loss function

In [39]:
# loss_func = nn.MSELoss(reduction='sum')
loss_func = nn.CrossEntropyLoss()

+ The optimizer

In [40]:
from torch import optim

# optimizer = optim.RMSprop(params=logistic_reg.parameters(), lr=1e-3)
optimizer = optim.SGD(logistic_reg.parameters(), lr=1e-3)

+ training Model

In [41]:
from tqdm import tqdm

train_proc = tqdm(range(10000))
for t in train_proc:
    logistic_reg.train()
    optimizer.zero_grad()

    y_pred = logistic_reg(X_train)
    print(y_pred)
    print(y_train)
    loss = loss_func(y_pred, y_train)
    train_proc.set_description('loss: %.4f' % loss.item())

    loss.backward()
    optimizer.step()

loss: 0.7409:   0%|          | 3/10000 [00:00<07:42, 21.63it/s]

tensor([[0.4874, 0.4009],
        [0.5266, 0.4467],
        [0.4765, 0.3974],
        [0.3579, 0.4012],
        [0.3618, 0.4731],
        [0.3545, 0.4117],
        [0.3404, 0.4806],
        [0.3364, 0.5384],
        [0.5335, 0.4001],
        [0.3278, 0.4457],
        [0.3536, 0.4213],
        [0.2559, 0.4452],
        [0.5261, 0.4938],
        [0.4609, 0.4290],
        [0.5025, 0.4039],
        [0.3466, 0.5154],
        [0.2500, 0.5025],
        [0.4675, 0.4601],
        [0.4956, 0.4506],
        [0.2779, 0.4591],
        [0.3466, 0.5278],
        [0.3475, 0.5056],
        [0.3213, 0.5404],
        [0.4373, 0.5112],
        [0.4155, 0.4298],
        [0.5266, 0.4467],
        [0.4719, 0.4847],
        [0.4332, 0.4384],
        [0.5055, 0.4093],
        [0.3204, 0.5502],
        [0.4678, 0.3829],
        [0.5335, 0.4001],
        [0.2977, 0.5028],
        [0.2723, 0.5068],
        [0.2969, 0.5126],
        [0.3683, 0.4884],
        [0.4442, 0.4635],
        [0.4986, 0.4562],
        [0.3

loss: 0.7407:   0%|          | 9/10000 [00:00<07:34, 21.96it/s]

tensor([[0.4876, 0.4007],
        [0.5265, 0.4468],
        [0.4765, 0.3975],
        [0.3581, 0.4009],
        [0.3621, 0.4727],
        [0.3545, 0.4116],
        [0.3405, 0.4804],
        [0.3365, 0.5382],
        [0.5334, 0.4002],
        [0.3280, 0.4453],
        [0.3536, 0.4212],
        [0.2561, 0.4448],
        [0.5261, 0.4938],
        [0.4608, 0.4290],
        [0.5024, 0.4040],
        [0.3468, 0.5151],
        [0.2501, 0.5023],
        [0.4674, 0.4602],
        [0.4954, 0.4508],
        [0.2781, 0.4587],
        [0.3468, 0.5275],
        [0.3477, 0.5052],
        [0.3216, 0.5400],
        [0.4373, 0.5112],
        [0.4157, 0.4295],
        [0.5265, 0.4468],
        [0.4719, 0.4847],
        [0.4331, 0.4384],
        [0.5054, 0.4093],
        [0.3207, 0.5498],
        [0.4677, 0.3830],
        [0.5334, 0.4002],
        [0.2979, 0.5024],
        [0.2726, 0.5064],
        [0.2971, 0.5123],
        [0.3684, 0.4883],
        [0.4441, 0.4635],
        [0.4984, 0.4562],
        [0.3

loss: 0.7405:   0%|          | 15/10000 [00:00<07:12, 23.09it/s]

...

loss: 0.7391:   1%|          | 54/10000 [00:02<07:21, 22.54it/s]

tensor([[0.4891, 0.3988],
        [0.5252, 0.4479],
        [0.4758, 0.3978],
        [0.3605, 0.3978],
        [0.3644, 0.4695],
        [0.3549, 0.4107],
        [0.3420, 0.4783],
        [0.3379, 0.5362],
        [0.5322, 0.4011],
        [0.3303, 0.4421],
        [0.3537, 0.4206],
        [0.2580, 0.4418],
        [0.5253, 0.4943],
        [0.4600, 0.4296],
        [0.5012, 0.4049],
        [0.3487, 0.5124],
        [0.2516, 0.4996],
        [0.4665, 0.4608],
        [0.4942, 0.4518],
        [0.2805, 0.4550],
        [0.3487, 0.5249],
        [0.3499, 0.5023],
        [0.3242, 0.5364],
        [0.4367, 0.5114],
        [0.4175, 0.4273],
        [0.5252, 0.4479],
        [0.4715, 0.4848],
        [0.4326, 0.4385],
        [0.5045, 0.4098],
        [0.3231, 0.5465],
        [0.4670, 0.3834],
        [0.5322, 0.4011],
        [0.3003, 0.4989],
        [0.2749, 0.5028],
        [0.2992, 0.5090],
        [0.3690, 0.4873],
        [0.4436, 0.4635],
        [0.4975, 0.4569],
        [0.3

loss: 0.7390:   1%|          | 58/10000 [00:02<07:22, 22.48it/s]

tensor([[0.4892, 0.3986],
        [0.5251, 0.4480],
        [0.4757, 0.3978],
        [0.3608, 0.3974],
        [0.3647, 0.4692],
        [0.3550, 0.4106],
        [0.3421, 0.4780],
        [0.3380, 0.5360],
        [0.5321, 0.4012],
        [0.3306, 0.4417],
        [0.3537, 0.4206],
        [0.2582, 0.4415],
        [0.5252, 0.4944],
        [0.4599, 0.4297],
        [0.5011, 0.4050],
        [0.3489, 0.5122],
        [0.2518, 0.4994],
        [0.4664, 0.4609],
        [0.4941, 0.4519],
        [0.2808, 0.4546],
        [0.3489, 0.5246],
        [0.3501, 0.5019],
        [0.3245, 0.5360],
        [0.4367, 0.5114],
        [0.4177, 0.4270],
        [0.5251, 0.4480],
        [0.4714, 0.4848],
        [0.4325, 0.4385],
        [0.5044, 0.4099],
        [0.3233, 0.5461],
        [0.4669, 0.3834],
        [0.5321, 0.4012],
        [0.3006, 0.4985],
        [0.2752, 0.5024],
        [0.2995, 0.5087],
        [0.3691, 0.4872],
        [0.4436, 0.4635],
        [0.4974, 0.4569],
        [0.3

KeyboardInterrupt: 


+ Model Performance


In [ ]:
import torch

with torch.no_grad():
    logits = logistic_reg(X_test)
    y_pred = nn.Softmax(dim=1)(logits)
    y_predicted_cls = y_pred.argmax(1)
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 1.0000


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test.cpu(), y_predicted_cls.cpu()))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         7

    accuracy                           1.00        13
   macro avg       1.00      1.00      1.00        13
weighted avg       1.00      1.00      1.00        13



### Exercise 2  Handwriting recognition with MLP

Like last week's lab , your task in this section is also about recognizing handwritten digits, but you are required to use MLP to complete the exercise. It is recommended that you define an MLP class, which is a subclass of `nn.module`.

<font color='red' size=4>Note that your accuracy in this section will directly determine your score.</font>

For this exercise we use the `minist` dataset.

+ load datasets

In [ ]:
import struct
import numpy as np
from array import array

def read_images_labels(images_filepath, labels_filepath):        
    labels, images = [], []

    with open(labels_filepath, 'rb') as file:
        magic, size = struct.unpack('>II', file.read(8))
        assert magic == 2049
        labels = array('B', file.read())        

    with open(images_filepath, 'rb') as file:
        magic, size, rows, cols = struct.unpack('>IIII', file.read(16))
        assert magic == 2051
        image_data = array('B', file.read())

    for i in range(size):
        images.append([0] * rows * cols)
    for i in range(size):
        img = np.array(image_data[i * rows * cols:(i + 1) * rows * cols])
        images[i][:] = img

    return np.array(images), np.array(labels)

In [ ]:
X_train, y_train = read_images_labels('datasets/MNIST/raw/train-images-idx3-ubyte', 'datasets/MNIST/raw/train-labels-idx1-ubyte')
X_test, y_test = read_images_labels('datasets/MNIST/raw/t10k-images-idx3-ubyte', 'datasets/MNIST/raw/t10k-labels-idx1-ubyte')

X_train = from_numpy(np.array(X_train)).float().to('cpu')
y_train = from_numpy(np.array(y_train)).long().to('cpu')
X_test = from_numpy(np.array(X_test)).float().to('cpu')
y_test = from_numpy(np.array(y_test)).long().to('cpu')

+ Define a MLP subclass of nn. Module

In [ ]:
from torch import nn

class MLP(nn.Module):
    def __init__(self, n_i, n_h, n_o):
        super(MLP, self).__init__()
        self.model = nn.Sequential(
            nn.Flatten(),
            nn.Linear(n_i, n_h),
            nn.ReLU(),
            nn.Linear(n_h, n_o),
        )

    def forward(self, x):
        return self.model.forward(x)

+ Create the model

In [ ]:
mlp = MLP(X_train.shape[1], X_train.shape[1]//2, y_train.unique().size()[0]).to('cpu')
mlp.requires_grad_()

MLP(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=392, bias=True)
    (2): ReLU()
    (3): Linear(in_features=392, out_features=10, bias=True)
  )
)

 + Loss function

In [ ]:
criterions = nn.CrossEntropyLoss()

+ The optimizer

In [ ]:
optimizer = optim.SGD(mlp.parameters(), lr=1e-3)

+ training Model

In [ ]:
train_proc = tqdm(range(10000))

for t in train_proc:
    mlp.train()
    optimizer.zero_grad()

    y_pred = mlp(X_train)
    loss = criterions(y_pred, y_train)
    train_proc.set_description('loss: %.4f' % loss.item())

    loss.backward()
    optimizer.step()

loss: 0.0306:  31%|███       | 3108/10000 [04:55<10:55, 10.52it/s]


KeyboardInterrupt: 

+ Model Performance

In [ ]:
with torch.no_grad():
    logits = mlp(X_test)
    y_pred = nn.Softmax(dim=1)(logits)
    y_predicted_cls = y_pred.argmax(1)
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.10f}')

accuracy: 0.9575999975


In [ ]:
# classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, y_predicted_cls))

              precision    recall  f1-score   support

           0       0.98      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.96      0.96      0.96      1032
           3       0.95      0.95      0.95      1010
           4       0.96      0.96      0.96       982
           5       0.95      0.93      0.94       892
           6       0.96      0.96      0.96       958
           7       0.96      0.96      0.96      1028
           8       0.93      0.94      0.94       974
           9       0.94      0.94      0.94      1009

    accuracy                           0.96     10000
   macro avg       0.96      0.96      0.96     10000
weighted avg       0.96      0.96      0.96     10000



### Exercise 3  Questions (10 points )
1.What's the difference between logistic regression and Perceptron?

- Logistic regression uses a logistic function, while perceptron uses a step function. Therefore the output of logistic regression is continuous, while the output of perceptron is discrete.
- The parameter vector for the perceptron may be arbitrarily scaled compared to the one derived by logistic regression. Any scaling of the parameter vector will define the same boundary, but the probabilities calculated by logistic regression depend on the exact scaling.
> ref: [stackoverflow](https://stats.stackexchange.com/questions/162257/whats-the-difference-between-logistic-regression-and-perceptron)

2.Advantages and disadvantages of neural networks?

- Advantages
  - it can learn continuously, for specific users/tasks, the performance can be improved over time
  - easy to use, no need to manually design features
- Disadvantages
  - it's interpretation is low, say, hard to explain the reason why the model makes a decision
  - easy to overfit
  - requires a large dataset to train

3.What is the role of Activation Function in Neural networks?

It's the import role that makes the NN to be non-linear, and thus the system could be more powerful. 
It also decides whether a neuron should be activated or not.